### TP : Classification de texte avec un RNN

Dans ce TP, nous allons construire un modèle de réseau de neurones récurrent (RNN) pour effectuer une analyse de sentiments sur le jeu de données IMDB. Nous suivrons les étapes suivantes :

1. Préparation de l'environnement et des données.
2. Création de l'encodeur de texte.
3. Construction et entraînement du modèle RNN.
4. Évaluation et visualisation des performances.

### Étape 1 : Préparation de l'environnement et des données

Importez les bibliothèques nécessaires (`numpy`, `tensorflow`, `tensorflow_datasets`, `matplotlib`).

In [ ]:
# Votre code ici

Chargez le jeu de données IMDB via `tensorflow_datasets` et séparez-le en ensembles d'entraînement et de test.

In [ ]:
# Votre code ici

Afficher un échantillon de données

In [ ]:
# Votre code ici

La cellule suivante prépare les ensembles de données en les mélangeant, les groupant en batchs, et en activant la prélecture pour optimiser les performances.

Pourquoi utiliser .prefetch ? <br>
Lors de l'entraînement d'un modèle, les étapes de traitement des données (chargement, déchiffrement, augmentation, etc.) peuvent devenir un goulot d'étranglement si elles prennent plus de temps que l'entraînement lui-même.<br>
Sans prefetch : Le modèle attend que les données soient préparées avant de continuer son entraînement.<br>
Avec prefetch : Les données pour la prochaine itération sont préparées en parallèle pendant que le modèle entraîne l'itération en cours.

Que fait tf.data.AUTOTUNE ?<br>
Le paramètre AUTOTUNE permet à TensorFlow de déterminer automatiquement le nombre optimal de lots de données à précharger dans le buffer, en fonction des ressources disponibles (CPU, GPU, mémoire).

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Exemple d'affichage
for example, label in train_dataset.take(1):
    print('Exemples de textes :', example.numpy()[:3])
    print('Labels correspondants :', label.numpy()[:3])

### Étape 2 : Création de l'encodeur de texte

Utilisez `TextVectorization` de Keras pour encoder les textes en indices numériques. Adaptez l'encodeur aux données d'entraînement pour construire le vocabulaire. Note : dans TensorFlow Keras, fit est réservé à l'entraînement des modèles. Son pendant est adapt lorsqu'il concerne des couches qui "s'ajustent" aux données (prétraitement).


In [ ]:
# Votre code ici

Affichez les 20 premiers mots du vocabulaire pour validation.

In [ ]:
# Votre code ici

### Étape 3 : Construction du modèle RNN

Construisez un modèle séquentiel Keras composé des couches suivantes :
  1. L'encodeur de texte.
  2. Une couche d'embedding avec 64 dimensions.
  3. Une couche bidirectionnelle avec un LSTM de 64 unités.
  4. Une couche Dense avec 64 unités et activation ReLU.
  5. Une couche Dense avec une sortie unique (classification binaire).

In [ ]:
# Votre code ici

Compilez le modèle avec une perte de votre choix (bien choisie !), l'optimiseur `Adam` et la métrique `accuracy`.

In [ ]:
# Votre code ici

Imaginons que nous n'ayons pas renseigné d'argument activation dans la dernière couche dense. Quel va être l'impact, s'il y en a un, par rapport au scénario où nous aurions renseigné "sigmoid" ? Proposer une règle pour convertir les outputs en prédictions binaires. 

### Étape 4 : Entraînement du modèle

Entraînez le modèle sur le jeu d'entraînement pour 10 epochs.

In [ ]:
# Votre code ici

### Etape 5 : Evaluation du modèle et affichage des résultats

Évaluez le modèle sur le jeu de test.
Affichez les courbes d'entraînement (précision et perte).

In [ ]:
# Votre code ici

In [ ]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_' + metric], '')
    plt.xlabel("Époques")
    plt.ylabel(metric)
    plt.legend([metric, 'val_' + metric])

In [ ]:
# Votre code ici

### Étape 6 : Prédiction sur de nouveaux textes

Effectuez une prédiction sur un texte d'exemple. Ajoutez un second texte rempli pour vérifier l'effet du padding sur les prédictions.

In [ ]:
# Votre code ici

### Étape 7 : Ajouter une couche LSTM supplémentaire

Ajoutez une seconde couche bidirectionnelle LSTM (avec `return_sequences=True` sur la première). Réentraînez le modèle et observez les performances.

**A noter :** La sortie de la première couche LSTM est une séquence qui peut être traitée par la seconde couche.

In [ ]:
# Votre code ici